In [1]:
import cv2 as cv
import numpy as np

In [2]:
img_fn = ["2/img-1.tif", "2/img-2.tif", "2/img-3.tif", "2/img-4.tif", "2/img-5.tif", "2/img-6.tif", "2/img-7.tif", "2/img-8.tif", "2/img-9.tif"]
img_list = [cv.imread(fn) for fn in img_fn]
exposure_times = np.array([(1/8000), (1/4000), (1/2000), (1/1000), (1/500), (1/250), (1/125), (1/60), (1/30)], dtype=np.float32)

### Register images

In [ ]:
gray = [cv.cvtColor(img, cv.COLOR_RGB2GRAY) for img in img_list]
norm = [cv.equalizeHist(test) for test in gray]
warp_mode = cv.MOTION_EUCLIDEAN
criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 100, 1e-10)

output = []
for i in range(len(norm) - 1):
    print("Working on " + str(i+ 1) + " of " + str(len(norm) - 1))
    warp_matrix = np.eye(2, 3, dtype=np.float32)
    (_, warp_matrix) = cv.findTransformECC(norm[i+1], norm[0], warp_matrix, warp_mode, criteria)
    aligned_image = cv.warpAffine(img_list[i+1], warp_matrix[:2], (img_list[i+1].shape[1], img_list[i+1].shape[0]), flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP)

    output += [aligned_image]

Working on 1 of 9
Working on 2 of 9
Working on 3 of 9
Working on 4 of 9
Working on 5 of 9
Working on 6 of 9
Working on 7 of 9
Working on 8 of 9


In [4]:
registered = [img.astype('uint8') for img in output]

In [21]:
test = [img_list[0]] + output

In [22]:
merge_debevec = cv.createMergeDebevec()
hdr_debevec = merge_debevec.process(test, times=exposure_times.copy())

In [23]:
tonemap1 = cv.createTonemap(gamma=2.2)
res_debevec = tonemap1.process(hdr_debevec.copy())

In [24]:
res_debevec_8bit = np.clip(res_debevec*255, 0, 255).astype('uint8')

C:\Users\Alex\AppData\Local\Temp\ipykernel_31188\2826973509.py:1: RuntimeWarning: invalid value encountered in cast
  res_debevec_8bit = np.clip(res_debevec*255, 0, 255).astype('uint8')


In [25]:
cv.imwrite("ldr_debevec.jpg",
 res_debevec_8bit)

True